# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.232993692324                   -0.50    7.0         
  2   -7.249581092803       -1.78       -1.40    1.0   7.01ms
  3   -7.251072239149       -2.83       -1.88    2.0   8.26ms
  4   -7.251011770409   +   -4.22       -1.90    1.0   6.99ms
  5   -7.251329914374       -3.50       -2.72    1.0   50.3ms
  6   -7.251337729214       -5.11       -3.13    2.0   8.07ms
  7   -7.251338701750       -6.01       -3.57    2.0   8.57ms
  8   -7.251338788850       -7.06       -4.13    1.0   7.22ms
  9   -7.251338796977       -8.09       -4.47    2.0   8.74ms
 10   -7.251338798470       -8.83       -5.00    1.0   7.24ms
 11   -7.251338798695       -9.65       -5.51    2.0   8.96ms
 12   -7.251338798702      -11.16       -5.87    1.0   7.42ms
 13   -7.251338798704      -11.65       -6.26    2.0   8.77ms
 14   -7.251338798705      -12.34       -6.93    2.0   8.87ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05786104761270987
[ Info: Arnoldi iteration step 2: normres = 0.7212084074816676
[ Info: Arnoldi iteration step 3: normres = 0.655478199457947
[ Info: Arnoldi iteration step 4: normres = 0.21694125027914068
[ Info: Arnoldi iteration step 5: normres = 0.3273328140708042
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (8.74e-03, 4.69e-02, 2.55e-01, 1.99e-01, 4.99e-03)
[ Info: Arnoldi iteration step 6: normres = 0.5660839287260718
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (3.93e-03, 1.36e-01, 5.04e-01, 1.34e-01, 1.69e-01)
[ Info: Arnoldi iteration step 7: normres = 0.07299570772088708
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.72e-04, 3.34e-02, 1.19e-02, 4.40e-02, 3.36e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09789435002333843
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (7.31e-0